# MotionDataset2label with ResNet

In [1]:
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt/code")' Batcher ModelSupport Datasets ImageClassificationModels

Installing packages:
	.package(path: "/notebooks/language2motion.gt/code")
		Batcher
		ModelSupport
		Datasets
		ImageClassificationModels
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmpl511ldfg/swift-install
[1/2] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
Initializing Swift...
Installation complete!


In [2]:
import Foundation
import TensorFlow
import PythonKit

import Batcher
import ModelSupport
import Datasets
import ImageClassificationModels

In [3]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

('inline', 'module://ipykernel.pylab.backend_inline')


# 1-channel ResNet model

In [4]:
var model = ResNet(classCount: 5, depth: .resNet18, downsamplingInFirstStage: false, channelCount: 1)

In [5]:
let optimizer = SGD(for: model, learningRate: 0.001)

# load dataset

In [6]:
let batchSize = 25

let serializedDatasetURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/motion_dataset.motion_flag.normalized.100.plist")
let labelsURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/labels_ds_v2.csv")

let dataset = Motion2Label(
    batchSize: batchSize, 
    serializedDatasetURL: serializedDatasetURL,
    labelsURL: labelsURL
)
print("dataset.training.count: \(dataset.training.count)")
print("dataset.test.count: \(dataset.test.count)")

MotionData(motionSamples: 99)
trainTensorPairs.count = 79
testTensorPairs.count = 20
dataset.training.count: 4
dataset.test.count: 1


In [ ]:
Array<PartialKeyPath<TangentVector>>
Swift.WritableKeyPath<ImageClassificationModels.ResNet.TangentVector, ImageClassificationModels.ConvBN.TangentVector>
Swift.WritableKeyPath<ImageClassificationModels.ResNet.TangentVector, TensorFlow.EmptyTangentVector>
Swift.WritableKeyPath<ImageClassificationModels.ResNet.TangentVector, Swift.Array<ImageClassificationModels.ResidualBlock.TangentVector>.DifferentiableView>
Swift.WritableKeyPath<ImageClassificationModels.ResNet.TangentVector, TensorFlow.EmptyTangentVector>
Swift.WritableKeyPath<ImageClassificationModels.ResNet.TangentVector, TensorFlow.EmptyTangentVector>
Swift.WritableKeyPath<ImageClassificationModels.ResNet.TangentVector, TensorFlow.Dense<Swift.Float>.TangentVector>

In [ ]:
Array<PartialKeyPath<TangentVector>>
TangentVector
EmptyTangentVector
DifferentiableView
EmptyTangentVector
EmptyTangentVector
TangentVector

In [ ]:
Array<PartialKeyPath<TangentVector>>

In [16]:
print("Starting motion2label training...")

for _ in 1...10 {
//     print("epoch \(epoch)")
    Context.local.learningPhase = .training
    var trainingLossSum: Float = 0
    var trainingBatchCount = 0
    for batch in dataset.training.sequenced() {
//         print("progress \(100.0*Float(trainingBatchCount)/Float(dataset.training.count))%")
        let (tensors, labels) = (batch.first, batch.second)
        let (loss, gradients) = valueWithGradient(at: model) { model -> Tensor<Float> in
            let logits = model(tensors)
            return softmaxCrossEntropy(logits: logits, labels: labels)
        }
        trainingLossSum += loss.scalarized()
        trainingBatchCount += 1
        print(type(of:gradients.allKeyPaths))
        for kp in gradients.allKeyPaths {
//             print(kp)
             let g = gradients[keyPath: kp]
//             let g: AnyObject = gradients[keyPath: kp] as AnyObject
//             print(type(of:g))
            
//             if g.isKind(of:Collection) {
                
//             }
//             print((g as! Collection).count)
        }
//         print(gradients.shape)

//         for kp in gradients.allWritableKeyPaths(to: TensorFlow.Dense<Swift.Float>.TangentVector.self) {
        for kp in gradients.allKeyPaths(to: ImageClassificationModels.ResNet.TangentVector.self) {
            let g: ImageClassificationModels.ResNet.TangentVector = gradients[keyPath: kp] as  ImageClassificationModels.ResNet.TangentVector
//             print(type(of:g.weight))
//             print(g.weight.scalarCount)
//             print(g.count)
        }

        optimizer.update(&model, along: gradients)
    }

//     Context.local.learningPhase = .inference
//     var testLossSum: Float = 0
//     var testBatchCount = 0
//     var correctGuessCount = 0
//     var totalGuessCount = 0
//     for batch in dataset.test.sequenced() {
// //         print("batch")
//         let (tensors, labels) = (batch.first, batch.second)
//         let logits = model(tensors)
//         testLossSum += softmaxCrossEntropy(logits: logits, labels: labels).scalarized()
//         testBatchCount += 1

//         let correctPredictions = logits.argmax(squeezingAxis: 1) .== labels
//         correctGuessCount = correctGuessCount
//             + Int(
//                 Tensor<Int32>(correctPredictions).sum().scalarized())
//         totalGuessCount = totalGuessCount + batchSize
//     }

//     let accuracy = Float(correctGuessCount) / Float(totalGuessCount)
//     print(
//         """
//         [Epoch \(epoch)] \
//         Training loss: \(trainingLossSum  / Float(trainingBatchCount)) \
//         Accuracy: \(correctGuessCount)/\(totalGuessCount) (\(accuracy*100)%) \
//         Loss: \(testLossSum / Float(testBatchCount))
//         """
//     )
}

: 